In [1]:
!pip install yfinance pandas numpy tqdm


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime


In [5]:
import pandas as pd
import requests
from io import StringIO

sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(sp500_url, headers=headers)
response.raise_for_status()

html_data = StringIO(response.text)

sp500_table = pd.read_html(html_data)[0]

tickers = sp500_table['Symbol'].tolist()
tickers = [ticker.replace('.', '-') for ticker in tickers]

print("Total Tickers:", len(tickers))


Total Tickers: 503


In [9]:
above_200ma_count = 0
valid_stocks = 0

for ticker in tqdm(tickers):
    try:
        data = yf.download(
            ticker,
            period="300d",
            progress=False,
            auto_adjust=True
        )

        if data.empty or len(data) < 200:
            continue

        close = data['Close']

        ma200 = close.rolling(200).mean()

        latest_close = float(close.iloc[-1])
        latest_ma200 = float(ma200.iloc[-1])

        if not np.isnan(latest_ma200):
            valid_stocks += 1

            if latest_close > latest_ma200:
                above_200ma_count += 1

    except Exception as e:
        print("Error in:", ticker, e)

if valid_stocks > 0:
    breadth_percentage = (above_200ma_count / valid_stocks) * 100
else:
    breadth_percentage = 0

print("Valid Stocks:", valid_stocks)
print("Above 200MA:", above_200ma_count)
print("Market Breadth:", round(breadth_percentage, 2), "%")


  0%|          | 0/503 [00:00<?, ?it/s]/tmp/ipython-input-562870453.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  latest_close = float(close.iloc[-1])
/tmp/ipython-input-562870453.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  latest_ma200 = float(ma200.iloc[-1])
  0%|          | 1/503 [00:00<02:15,  3.71it/s]/tmp/ipython-input-562870453.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  latest_close = float(close.iloc[-1])
/tmp/ipython-input-562870453.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  latest_ma200 = float(ma200.iloc[-1])
  0%|          | 2/503 [00:00<01:52,  4.46

Valid Stocks: 502
Above 200MA: 335
Market Breadth: 66.73 %


In [11]:
import yfinance as yf
import numpy as np
import pandas as pd

spy = yf.download("SPY", period="1y", auto_adjust=True)
tlt = yf.download("TLT", period="1y", auto_adjust=True)

spy['log_return'] = np.log(spy['Close'] / spy['Close'].shift(1))
tlt['log_return'] = np.log(tlt['Close'] / tlt['Close'].shift(1))

merged = pd.DataFrame({
    'spy_ret': spy['log_return'],
    'tlt_ret': tlt['log_return']
}).dropna()

rolling_corr = merged['spy_ret'].rolling(60).corr(merged['tlt_ret'])

latest_corr = rolling_corr.iloc[-1]

print("Latest 60-Day Correlation:", round(latest_corr, 3))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Latest 60-Day Correlation: 0.151


In [12]:
# Breadth Status
if breadth_percentage > 65:
    breadth_status = "Green"
elif breadth_percentage > 40:
    breadth_status = "Yellow"
else:
    breadth_status = "Red"

# Diversification Warning
div_warning = latest_corr > 0.8

# System Risk Logic
if breadth_percentage < 40 or latest_corr > 0.8:
    system_risk = "Red"
elif breadth_percentage <= 65:
    system_risk = "Yellow"
else:
    system_risk = "Green"

print("\n--- SYSTEM HEALTH ---")
print("Breadth Status:", breadth_status)
print("Diversification Warning:", div_warning)
print("System Risk:", system_risk)



--- SYSTEM HEALTH ---
Breadth Status: Green
Diversification Warning: False
System Risk: Green


In [14]:
macro_health = {
    "date": str(datetime.today().date()),
    "breadth_percentage": round(breadth_percentage, 2),
    "correlation": round(float(latest_corr), 3),
    "diversification_warning": bool(div_warning),
    "system_risk": system_risk
}

macro_df = pd.DataFrame([macro_health])

macro_df.to_json("macro_health.json", orient="records", indent=4)

macro_df


,date,breadth_percentage,correlation,diversification_warning,system_risk
0,2026-02-16,66.73,0.151,False,Green


In [15]:
from google.colab import files
files.download("macro_health.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>